In [ ]:
#| default_exp memory

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [8]:
#| export
import subprocess
import psutil
import tsai.imports as ts

## Check GPU memory usage

In [9]:
#| export
def get_gpu_memory(device=0):
    total_memory = subprocess.check_output(["nvidia-smi", "--query-gpu=memory.total", "--format=csv,noheader,nounits", "--id=" + str(device)])
    total_memory = int(total_memory.decode().split('\n')[0])
    used_memory = subprocess.check_output(["nvidia-smi", "--query-gpu=memory.used", "--format=csv,noheader,nounits",  "--id=" + str(device)])
    used_memory = int(used_memory.decode().split('\n')[0])

    percentage = round((used_memory / total_memory) * 100)
    return used_memory, total_memory, percentage

In [13]:
#| export
def color_for_percentage(percentage):
    if percentage < 20:
        return "\033[90m"  # Gray
    elif percentage < 40:
        return "\033[94m"  # Blue
    elif percentage < 60:
        return "\033[92m"  # Green
    elif percentage < 80:
        return "\033[93m"  # Orange
    else:
        return "\033[91m"  # Red

In [14]:
#| export
def create_bar(percentage, color_code, length=20):
    filled_length = int(length * percentage // 100)
    bar = "█" * filled_length + "-" * (length - filled_length)
    return color_code + bar + "\033[0m"  # Apply color and reset after bar

In [15]:
#| export
def gpu_memory_status(device=0):
    used, total, percentage = get_gpu_memory(device)
    color_code = color_for_percentage(percentage)
    bar = create_bar(percentage, color_code)
    print(f"GPU | Used mem: {used}")
    print(f"GPU | Used mem: {total}")
    print(f"GPU | Memory Usage: [{bar}] {color_code}{percentage}%\033[0m")

In [16]:
#| hide
gpu_memory_status()
ts.get_gpu_memory()

GPU | Used mem: 3
GPU | Used mem: 24576
GPU | Memory Usage: [--------------------] 0%


[24.0, 24.0, 24.0]

In [17]:
#| hide 
ts.my_setup()

os              : Linux-5.15.0-97-generic-x86_64-with-glibc2.31
python          : 3.10.13
tsai            : 0.3.9
fastai          : 2.7.14
fastcore        : 1.5.29
torch           : 1.13.0
device          : 3 gpus (['NVIDIA GeForce RTX 3090', 'NVIDIA GeForce RTX 3090', 'NVIDIA GeForce RTX 3090'])
cpu cores       : 24
threads per cpu : 2
RAM             : 251.54 GB
GPU memory      : [24.0, 24.0, 24.0] GB


## Check CPU memory usage

In [ ]:
#| export
def get_cpu_memory():
    mem = psutil.virtual_memory()
    total_memory = mem.total // (1024**2)  # Convertir a MB
    used_memory = (mem.total - mem.available) // (1024**2)  # Convertir a MB
    percentage = mem.percent
    return used_memory, total_memory, percentage

In [ ]:
#| export
def cpu_memory_status():
    used, total, percentage = get_cpu_memory()
    color_code = color_for_percentage(percentage)
    bar = create_bar(percentage, color_code)
    print(f"CPU | Used mem: {used}")
    print(f"CPU | Used mem: {total}")
    print(f"CPU | Memory Usage: [{bar}] {color_code}{percentage}%\033[0m")

In [ ]:
#| hide
cpu_memory_status()